# Dev Setups: Connecting Python and SQL (Linux version)

The purpose of this IPython notebook is to demonstrate the usefulness of connecting python to a relational database by using a python toolkit called SQLAlchemy. This tutorial follows the previous document, *** Testing Python and Data Science basic stack ***

**This notebook provides Linux specific instructions. Updated 8/18**

### First off, what is a relational database?

Basically, it is a way to store information such that information can be retrieved from it.

MySQL and PostgreSQL are examples of relational databases.  For the purposes of an Insight project, you can use either one.

Why would you use a relational database instead of a csv or two?

**A few reasons:**

- They scale easily

-  They are easy to query

- It’s possible to do transactions in those cases where you need to write to a database, not just read from it

- Everyone in industry uses them, so you should get familiar with them, too.

***What does a relational database look like? ***

We can take a look.  First we need to set up a few things. The first thing we want to do is to get a PostgreSQL server up and running.  




## Postgres Linux installation
These instructions have been tested for installing Postgres 9.6 on Ubuntu 16.04.

Check out this [post](https://askubuntu.com/questions/831292/how-to-install-postgresql-9-6-on-any-ubuntu-version). This is for installing version 9.6, which should be fine for our purposes. Run the following series of commands in the terminal (Note: for Ubuntu 18.04, you should only need the first and last commands below):

    sudo add-apt-repository "deb http://apt.postgresql.org/pub/repos/apt/ xenial-pgdg main"
    wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
    sudo apt-get update
    sudo username = 'postgres'
    sudo apt-get install postgresql-9.6

Postgres 9.6 should now be installed. Start the service.  

    sudo service postgresql start
    
You can always check that the service is running by: 

    sudo service postgresql status
    
Set Unix password for user `postgres`

    sudo passwd postgres

Set the super user to `postgres`.

    su postgres
    
You can now launch the Postgres interactivate terminal `psql`, where you'll enter as user `postgres`.

    psql
    
Inside `psql`, a list of users can be accessed by running the query:

    SELECT username FROM pg_user;
 
Set the password for user `postgres`.

    \password postgres
    
To exit the terminal:

    \q

We'll come back to PostgreSQL in a moment. First, we'll set up SQLAlchemy.

<a id='python'></a>
## Install and load python packages

Inside your conda environment, install the necessary packages for python to talk to a sql database.

    conda install sqlalchemy psycopg2  
    pip3 install sqlalchemy_utils

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

## Create a database

In [ ]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username/password, and connection specifics
username = 'postgres'
password = 'my_password'     # change this
host     = 'localhost'
port     = '5432'            # default port that postgres listens on
db_name  = 'birth_db'

In [ ]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine( 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, db_name) )
print(engine.url)

In [ ]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

In [ ]:
# read a database from CSV and load it into a pandas dataframe
birth_data = pd.read_csv('births2012_downsampled.csv', index_col=0)

In [ ]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
birth_data.to_sql('birth_data_table', engine, if_exists='replace')

The above line (to_sql) is doing a lot of heavy lifting.  It's reading a dataframe, it's creating a table, and adding the data to the table.  So ** SQLAlchemy is quite useful! **

## Working with PostgresSQL without Python

As the postgres super user and enter the postgres terminal.

    su postgres
    psql

**Connect to the "birth_db" database we created**

    \c birth_db

**You should see something like the following**

`You are now connected to database "birth_db" as user "postgres".`


**Then try the following query:**

    SELECT * FROM birth_data_table;
    
Note that the semi-colon indicates an end-of-statement.

### You can see the table we created!  But it's kinda ugly and hard to read.

Try a few other sample queries.  Before you type in each one, ask yourself what you think the output will look like:

`SELECT * FROM birth_data_table WHERE infant_sex='M';`

`SELECT COUNT(infant_sex) FROM birth_data_table WHERE infant_sex='M';`

`SELECT COUNT(gestation_weeks), infant_sex FROM birth_data_table WHERE infant_sex = 'M' GROUP BY gestation_weeks, infant_sex;`

`SELECT gestation_weeks, COUNT(gestation_weeks) FROM birth_data_table WHERE infant_sex = 'M' GROUP BY gestation_weeks;`

All the above queries run, but they are difficult to visually inspect in the Postgres terminal.

## Working with PostgreSQL in Python

In [ ]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = db_name, user = username, host = host, password = password )

Note: If you have trouble with the command above, make sure host is pointing to '/var/run/postgresql' and not tmp. If you're still having trouble, you may need to change the permissions in pg_hba.conf from peer to trust. 

In [ ]:
# query:
sql_query = """
SELECT * FROM birth_data_table WHERE delivery_method='Cesarean';
"""
birth_data_from_sql = pd.read_sql_query(sql_query,con)
birth_data_from_sql.head()

Once the data has been pulled into python, we can leverage pandas methods to work with the data.

In [ ]:
%matplotlib inline
birth_data_from_sql.hist(column='birth_weight');

### Is reading from a SQL database faster than from a Pandas dataframe?  Probably not for the amount of data you can fit on your machine.

In [ ]:
def get_data(sql_query, con):
    data = pd.read_sql_query(sql_query, con)
    return data

%timeit get_data(sql_query, con)

birth_data_from_sql = get_data(sql_query, con)
birth_data_from_sql.head()

In [ ]:
def get_pandas_data(df, col, value):
    sub_df = df.loc[(df[col] == value)]
    return sub_df

%timeit get_pandas_data(birth_data, 'delivery_method', 'Cesarean')

birth_data_out = get_pandas_data(birth_data, 'delivery_method', 'Cesarean')
birth_data_out.head()

This should have given you a quick taste of how to use SQLALchemy, as well as how to run a few SQL queries both at the command line and in python.  You can see that `pandas` is actually a quite a bit faster than PostgreSQL here. This is because we're working with quite a small database (2716 rows × 37 columns), and there is an overhead of time it takes to communicate between python and PostGreSQL.  But as your database gets bigger (and certainly when it's too large to store in memory), working with relational databases becomes a necessity.

#### Congrats! You now have Python and SQL ready to go!